In [4]:
from pulp import *
import pandas as pd

In [17]:
probA=LpProblem("Hornby", LpMinimize)
warehouse=['Atlanta','Buffalo','Chicago','Cincinnati','Detroit','Pittsburgh','Richmond','St Louis']
representative=['Atlanta','Birmingham','Buffalo','Charleston','Charlotte','Chattanooga','Chicago','Cincinnati','Cleveland','Columbus','Detroit','Evansville','Ft Wayne','Indianapolis','Knoxville','Louisville','Memphis','Nashville','Peoria','Pittsburgh','Richmond','St Louis']

trans_cost = [[1, 4, 13, 7, 4, 2, 16, 9, 13, 10, 15, 9, 14, 12, 4, 8, 8, 6, 16, 11, 8, 12],
    [13, 15, 1, 6, 10, 11, 10, 6, 3, 4, 4, 10, 8, 9, 9, 8, 14, 12, 11, 2, 7, 13],
    [16, 15, 10, 11, 15, 14, 1, 7, 7, 8, 6, 7, 2, 4, 12, 8, 12, 11, 2, 9, 14, 7],
    [11, 12, 8, 6, 10, 9, 9, 3, 6, 4, 8, 6, 7, 6, 7, 4, 10, 8, 10, 6, 9, 9],
    [15, 16, 4, 8, 13, 13, 6, 6, 3, 5, 1, 9, 4, 7, 11, 8, 14, 12, 8, 5, 11, 11],
    [11, 13, 4, 6, 8, 9, 9, 6, 5, 4, 7, 8, 9, 8, 9, 8, 12, 10, 11, 3, 7, 11],
    [8, 11, 7, 4, 4, 7, 14, 7, 8, 7, 11, 10, 12, 11, 6, 8, 12, 9, 15, 5, 1, 14],
    [12, 10, 13, 10, 14, 11, 7, 7, 10, 9, 11, 4, 6, 4, 9, 6, 6, 7, 5, 11, 14, 1]]

demand = [275, 160, 240, 260, 135, 160, 400, 200, 320, 220, 190, 100, 140, 310, 125, 340, 240, 210, 150, 340, 300, 260]

leasing_cost = [2700,2900,3500,2200,3300,3000,2000,1800]

min_amount = [200,150,250,200,200,200,150,200]

capacity = [1750,1250,2000,1500,1750,1500,1000,1500]

variable_cost = [6,8,9,7,8,8,6,5]
              
#print(trans_cost)
#print(demand)
#print(capacity)
#print(min_amount)
#print(leasing_cost)
#print(variable_cost)

# set up parameters
trans_cost = makeDict([warehouse,representative],trans_cost)
demand = dict(zip(representative,demand))
capacity = dict(zip(warehouse,capacity))
min_amount = dict(zip(warehouse,min_amount))
leasing_cost  = dict(zip(warehouse,leasing_cost))
variable_cost = dict(zip(warehouse,variable_cost))


# define decision variables
trans_vars = LpVariable.dicts("x", (warehouse, representative), lowBound =0, cat="Continuous")
lease_vars = LpVariable.dicts("y", warehouse,lowBound =0, cat="Binary" )

In [24]:
variable_cost

{'Atlanta': 6,
 'Buffalo': 8,
 'Chicago': 9,
 'Cincinnati': 7,
 'Detroit': 8,
 'Pittsburgh': 8,
 'Richmond': 6,
 'St Louis': 5}

In [25]:
# objective
probA +=lpSum([trans_cost[i][j]*trans_vars[i][j] for i in warehouse for j in representative])\
+lpSum([variable_cost[i]*trans_vars[i][j] for i in warehouse for j in representative])+ lpSum([leasing_cost[i]*lease_vars[i] for i in warehouse]) 

# Constraint1: demand at each representative
for j in representative:
    probA += lpSum([trans_vars[i][j] for i in warehouse]) >= demand[j],"representative_%s"%j

# Constraint2: warehouse Minimum allowed amount of furniture stored in each warehouse
for i in warehouse:
    probA += lpSum([trans_vars[i][j] for j in representative]) >= lease_vars[i]*min_amount[i],"warehouse_min%s"%i

# Constraint3: warehouse Maximum allowed amount of furnitur stored in each warehouse
for i in warehouse:
    probA += lpSum([trans_vars[i][j] for j in representative]) <= lease_vars[i]*capacity[i],"warehouse_max%s"%i
    
    
probA.solve()
print("Status:",LpStatus[probA.status])

Status: Optimal


In [26]:
print("Total cost =", value(probA.objective))

output=[]
for i in warehouse:
    var_output=[]
    for j in representative:
        var_output.append(trans_vars[i][j].varValue)
    var_output.append(lease_vars[i].varValue)
    output.append(var_output) 
representative.append('Lease')
output_df = pd.DataFrame(output,index=warehouse, columns=representative)
output_df

Total cost = 61475.0


,Atlanta,Birmingham,Buffalo,Charleston,Charlotte,Chattanooga,Chicago,Cincinnati,Cleveland,Columbus,...,Indianapolis,Knoxville,Louisville,Memphis,Nashville,Peoria,Pittsburgh,Richmond,St Louis,Lease
Atlanta,275.0,160.0,0.0,0.0,135.0,160.0,0.0,0.0,0.0,0.0,...,0.0,125.0,160.0,100.0,210.0,0.0,0.0,0.0,0.0,1
Buffalo,0.0,0.0,240.0,0.0,0.0,0.0,0.0,0.0,320.0,220.0,...,0.0,0.0,0.0,0.0,0.0,0.0,280.0,0.0,0.0,1
Chicago,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
Cincinnati,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
Detroit,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
Pittsburgh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
Richmond,0.0,0.0,0.0,260.0,0.0,0.0,0.0,200.0,0.0,0.0,...,0.0,0.0,180.0,0.0,0.0,0.0,60.0,300.0,0.0,1
St Louis,0.0,0.0,0.0,0.0,0.0,0.0,400.0,0.0,0.0,0.0,...,310.0,0.0,0.0,140.0,0.0,150.0,0.0,0.0,260.0,1


In [30]:
df

,Reference: Steco.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Unit Cost,NaN,Representative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Atlanta,Birmingham,Buffalo,Charleston,Charlotte,Chattanooga,Chicago,Cincinnati,...,Memphis,Nashville,Peoria,Pittsburgh,Richmond,St Louis,Fixed Cost,Variable Cost,Min\n(L),Max\n(U)
3,Warehouse,Atlanta,1,4,13,7,4,2,16,9,...,8,6,16,11,8,12,2700,6,200,1750
4,NaN,Buffalo,13,15,1,6,10,11,10,6,...,14,12,11,2,7,13,2900,8,150,1250
5,NaN,Chicago,16,15,10,11,15,14,1,7,...,12,11,2,9,14,7,3500,9,250,2000
6,NaN,Cincinnati,11,12,8,6,10,9,9,3,...,10,8,10,6,9,9,2200,7,200,1500
7,NaN,Detroit,15,16,4,8,13,13,6,6,...,14,12,8,5,11,11,3300,8,200,1750
8,NaN,Pittsburgh,11,13,4,6,8,9,9,6,...,12,10,11,3,7,11,3000,8,200,1500
9,NaN,Richmond,8,11,7,4,4,7,14,7,...,12,9,15,5,1,14,2000,6,150,1000
